# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `test2.dat` network as an example, which contains only 3 species and 2 reactions.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [1]:
from jaff.network import Network
from jaff.builder import Builder
import os

# Load the react_COthin network
network = Network("../networks/test2.dat")

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

Welcome to JAFF: Just Another Fancy Format!
Loading network from ../networks/test2.dat
Network label = test2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 525.37it/s]

2.0e-12*tgas**0.33
1.0e-12*exp(-1.222*av)
Variables found: ['av', 'tgas']
Loaded 2 reactions
Lodaded 0 photo-chemistry reactions
All done!
Network loaded: test2
Number of species: 3
Number of reactions: 2


## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [2]:
for i, species in enumerate(network.species):
    print(f"  {i}: {species.name} (mass: {species.mass:.2f}, charge: {species.charge})")

print(f"\nTotal species: {len(network.species)}")

  0: e- (mass: 0.00, charge: -1)
  1: CO+ (mass: 0.00, charge: 1)
  2: CO (mass: 0.00, charge: 0)

Total species: 3


In [3]:
# Show a few example reactions
for i, reaction in enumerate(network.reactions):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i+1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.rate}")
    print()

  1: e- + CO+ -> CO
      Rate: 2.0e-12*tgas**0.33

  2: CO -> CO+ + e-
      Rate: 1.0e-12*exp(-1.222*av)



## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [4]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builds_dir = builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print(f"Generated files are located at {builds_dir}")

Building network with template: kokkos_ode
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.hpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.cpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.cpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/CMakeLists.txt -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/CMakeLists.txt
Network built successfully using template 'kokkos_ode'.
Output files are located in: /Users/benwibking/amrex_codes/chemistry/jaff/examples

C++ Kokkos code generated successfully!
Generated files are located at /Users/benwibking/amrex_codes/chemistry/jaff/examples


## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [5]:
# List generated files
generated_files = [f for f in os.listdir(builds_dir) if f.endswith(('.cpp', '.hpp', '.txt'))]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

Generated files:
  CMakeLists.txt       (1,309 bytes)
  chemistry_ode.cpp    (5,998 bytes)
  chemistry_ode.hpp    (5,405 bytes)


## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [6]:
# Show the ChemistryODE class definition
with open(builds_dir + "/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, len(lines)):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")
        
        if '{' in line:
            brace_count += line.count('{')
        if '}' in line:
            brace_count -= line.count('}')
            if brace_count == 0:
                break

ChemistryODE class structure:
13: struct ChemistryODE {
14:     // Number of species in the chemical network
15:     // PREPROCESS_NUM_SPECIES
16: 
17:     static constexpr int neqs = 3;
18: 
19:     // PREPROCESS_END
20: 
21:     // Species indices
22:     // PREPROCESS_COMMONS
23: 
24:     static constexpr int idx_e = 0;
25:     static constexpr int idx_COj = 1;
26:     static constexpr int idx_CO = 2;
27:     static constexpr int nspecs = 3;
28:     static constexpr int nreactions = 2;
29: 
30:     // Common chemistry variables used in rate expressions
31:     // These should typically be passed as parameters or computed from the state
32:     static constexpr double DEFAULT_TEMPERATURE = 300.0;  // Default gas temperature in K
33:     static constexpr double DEFAULT_AV = 1.0;             // Default visual extinction
34:     static constexpr double DEFAULT_CRATE = 1.3e-17;      // Default cosmic ray ionization rate
35: 
36:     // Placeholder function for photorates (should be repla

In [7]:
# Show main function structure from chemistry_ode.cpp
with open(builds_dir + "/chemistry_ode.cpp", 'r') as f:
    lines = f.readlines()

# Find main function
main_start = None
for i, line in enumerate(lines):
    if "int main(" in line:
        main_start = i
        break

if main_start:
    print("Main function structure:")
    print("=" * 50)
    for i in range(main_start, len(lines)):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")

Main function structure:
41: int main(int argc, char* argv[]) {
42:     Kokkos::initialize(argc, argv);
43: 
44:     {
45:         using execution_space = Kokkos::DefaultExecutionSpace;
46:         using scalar_type = double;
47:         using vec_type = Kokkos::View<scalar_type*, execution_space>;
48:         using mat_type = Kokkos::View<scalar_type**, execution_space>;
49: 
50:         ChemistryODE mySys{};
51: 
52:         // Time integration parameters
53:         const scalar_type t_start = 0.0;
54:         scalar_type t_end = 3.15576e9; // 100 years in seconds default
55: 
56:         // Parse command line arguments
57:         if (argc > 1) {
58:             t_end = std::stod(argv[1]);
59:         }
60: 
61:         std::cout << "Solving Chemistry ODE System\n";
62:         std::cout << "Number of species: " << mySys.neqs << "\n";
63:         std::cout << "Number of reactions: " <<
64: // PREPROCESS_NUM_REACTIONS
65: 
66: 2
67: 
68: // PREPROCESS_END
69:          << "\n";
70:  

## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

* **Configure the build with CMake:**

In [8]:
!   cmake -S . -B build

-- Setting default Kokkos CXX standard to 17
-- Kokkos version: 4.7.0
-- The project name is: Kokkos
-- Using internal gtest for testing
-- Using -std=c++17 for C++17 standard as feature
-- Built-in Execution Spaces:
--     Device Parallel: NoTypeDefined
--     Host Parallel: NoTypeDefined
--       Host Serial: SERIAL
-- 
-- Architectures:
-- Using internal desul_atomics copy
-- Performing Test KOKKOS_LINK_OPTIONS_CHECK
-- Performing Test KOKKOS_LINK_OPTIONS_CHECK - Success
-- Experimental mdspan support is enabled
-- Using internal mdspan directory /Users/benwibking/amrex_codes/chemistry/jaff/examples/build/_deps/kokkos-src/tpls/mdspan/include
-- Kokkos Backends: SERIAL
-- The project name is: KokkosKernels

Kokkos Kernels version: 4.7.0
Kokkos Kernels ETI Types
   Devices:  <Serial,HostSpace>
   Scalars:  double
   Ordinals: int
   Offsets:  int
   Layouts:  LayoutLeft

Kokkos Kernels components
   COMMON:    ON
   BATCHED:   ON
   BLAS:      ON
   LAPACK:    ON
   GRAPH:     ON
   S

* **Build with parallel compilation:**

In [9]:
!   cmake --build build -j

[3/3] Linking CXX executable chemistry_odey_ode.dir/chemistry_ode.cpp.oake


* **Run the chemistry solver:**

In [10]:
!   ./build/chemistry_ode

Solving Chemistry ODE System
Number of species: 3
Number of reactions: 2
Time interval: [0, 3.15576e+09] seconds
Initial conditions:
  y[0] = 1e-06
  y[1] = 1e-10
  y[2] = 1e-10

Final solution at t = 3.15576e+09 seconds:
  y[0] = 1e-06
  y[1] = 1.00093e-10
  y[2] = 9.99071e-11

Conservation check:
  Initial sum: 1.0002e-06
  Final sum:   1.0002e-06
  Relative change: 0

Kokkos execution space: N6Kokkos6SerialE


* **Or, run with custom time interval (1e12 seconds):**

In [11]:
!   ./build/chemistry_ode 1e12

Solving Chemistry ODE System
Number of species: 3
Number of reactions: 2
Time interval: [0, 1e+12] seconds
Initial conditions:
  y[0] = 1e-06
  y[1] = 1e-10
  y[2] = 1e-10

Final solution at t = 1e+12 seconds:
  y[0] = 1.00002e-06
  y[1] = 1.24316e-10
  y[2] = 7.56836e-11

Conservation check:
  Initial sum: 1.00022e-06
  Final sum:   1.00022e-06
  Relative change: 0

Kokkos execution space: N6Kokkos6SerialE
